https://all-python.ru/osnovy/threading.html  
https://habr.com/ru/articles/804799/

In [1]:
import time
from time import sleep
from threading import Thread, Lock

In [2]:
def timer(func):
    def inner(*args, **kwargs):
        start = time.time()
        res = func(*args, **kwargs)
        elapsed = time.time() - start
        print(f'Time: {elapsed:.5f} sec ({func.__name__})')
        return res
    return inner

### Задача 1

In [3]:
# Функция, которая будет применена к каждому элементу
def square(n):
    return n * n

# Входной список значений
vals = [1, 2, 3, 4, 5]

In [7]:
@timer
def sync1(values: list):
    # Список для хранения результатов
    results = []

    def worker(value):
        result = square(value)
        results.append(result)
    
    for val in values:
        worker(val)
    
    print(results)

sync1(vals)

[1, 4, 9, 16, 25]
Time: 0.00000 sec (sync1)


In [8]:
@timer
def threaded1(values: list):
    # Список для хранения результатов
    results = []
    lock = Lock()  # Создаем объект блокировки
    
    # Функция для выполнения в потоке
    def worker(value):
        result = square(value)
        with lock:  # Блокируем доступ к общему ресурсу results
            results.append(result)
    
    # Создание и запуск потоков для вызова функции square
    threads = []
    for value in values:
        thread = Thread(target=worker, args=(value,))
        thread.start()
        threads.append(thread)
    
    # Ожидание завершения всех потоков
    for thread in threads:
        thread.join()
    
    print(results)

threaded1(vals)

[1, 4, 9, 16, 25]
Time: 0.00147 sec (threaded1)


### Вывод: CPU-bound операции в потоках выполняются чуть медленнее. чем в синхр.варианте

## Lock

### Race condition (гонка состояний) - ошибка при доступе к общим ресурсам

In [11]:
counter = 0

def increase(by):
    global counter

    local_counter = counter
    local_counter += by

    time.sleep(0.1)

    counter = local_counter
    print(f'{counter=}\n')


t1 = Thread(target=increase, args=(10,))
t2 = Thread(target=increase, args=(20,))

t1.start()
t2.start()

t1.join()
t2.join()

counter=10
counter=20




In [12]:
counter = 0

def increase(by, lock: Lock):
    global counter

    lock.acquire()

    local_counter = counter
    local_counter += by

    sleep(0.1)

    counter = local_counter
    print(f'{counter=}')

    lock.release()

lock = Lock()

t1 = Thread(target=increase, args=(10, lock,))
t2 = Thread(target=increase, args=(20, lock,))

t1.start()
t2.start()

t1.join()
t2.join()

counter=10
counter=30


#### Состояние гонки может приводить к различным проблемам:

* Утечки памяти.
* Потеря данных.
* Уязвимости в безопасности программы.
* Получение ошибочных результатов.
* Взаимные блокировки потоков.

### DeadLock

In [ ]:
a = 5
b = 10

a_lock = Lock()
b_lock = Lock()

def function_a():
    global a
    global b

    a_lock.acquire()
    print('Функция a, a_lock = заблокирован')
    sleep(1)
    b_lock.acquire()
    print('Функция a, b_lock = заблокирован')

    sleep(1)

    a_lock.release()
    print('Функция a, a_lock = разблокирован')
    b_lock.release()
    print('Функция a, b_lock = разблокирован')

def function_b():
    global a
    global b

    b_lock.acquire()
    print('Функция b, b_lock = заблокирован')
    a_lock.acquire()
    print('Функция b, a_lock = заблокирован')

    sleep(1)

    b_lock.release()
    print('Функция b, b_lock = разблокирован')
    a_lock.release()
    print('Функция b, a_lock = разблокирован')

t1 = Thread(target=function_a)
t2 = Thread(target=function_b)

t1.start()
t2.start()

t1.join()
t2.join()

print('Готово')

Функция a, a_lock = заблокирован
Функция b, b_lock = заблокирован


### Решение Deadlock

1. lock как контекстный менеджер 
```
lock = threading.Lock() 
with lock:
    # операторы
    pass
```

2. Использование try-finally
```
lock = threading.Lock()
lock.acquire()
try:
    # операторы
    pass
finally:
    lock.release()
```

3. Semathore

#### Semathore

In [72]:
import datetime
from threading import Semaphore

s = Semaphore(3)


def semaphore_func(value: int):
    s.acquire()
    now = datetime.datetime.now().strftime('%H:%M:%S')
    print(f'{now=}, {value=}')
    sleep(2)
    s.release()


threads = [Thread(target=semaphore_func, args=(i,)) for i in range(7)]

for t in threads:
    t.start()

for t in threads:
    t.join()

now='11:24:53', value=0
now='11:24:53', value=1
now='11:24:53', value=2
now='11:24:55', value=3
now='11:24:55', value=4
now='11:24:55', value=5
now='11:24:57', value=6


### Задача 2

In [5]:
import requests

urls = [
    'http://www.python.org',
    'https://docs.python.org/3/',
    'https://docs.python.org/3/whatsnew/3.7.html',
    'https://docs.python.org/3/tutorial/index.html',
    'https://docs.python.org/3/library/index.html',
    'https://docs.python.org/3/reference/index.html',
    'https://docs.python.org/3/using/index.html',
    'https://docs.python.org/3/howto/index.html',
    'https://docs.python.org/3/installing/index.html',
    'https://docs.python.org/3/distributing/index.html',
]

def info2(results):
    print(f'results: {len(results)}, total lenght: {sum([len(i) for i in results])}')

In [6]:
@timer
def sync2(_urls):
    results = []
    for url in _urls:
        with requests.Session() as session:
            src = session.get(url)
            results.append(src.content)
    return info2(results)

sync2(urls)

results: 10, total lenght: 603371
Time: 1.94346 sec (sync2)


In [8]:
@timer
def threaded2(_urls):
    results = []
    lock = Lock()  # Создаем объект блокировки
    
    # Функция для выполнения в потоке
    def worker(url):
        with requests.Session() as session:
            src = session.get(url)
            with lock:  # Блокируем доступ к общему ресурсу results
                results.append(src.content)
    
    # Создание и запуск потоков для вызова функции square
    threads = []
    for url in _urls:
        thread = Thread(target=worker, args=(url,))
        thread.start()
        threads.append(thread)
    
    # Ожидание завершения всех потоков
    for thread in threads:
        thread.join()

    return info2(results)

threaded2(urls)

results: 10, total lenght: 603371
Time: 0.29763 sec (threaded2)


### Пояснения

`threading.Thread()` - создает новый поток, пока неактивный  
`start()` - используется для запуска созданного потока  
`join()` - блокирует выполнение потока, который его вызвал, до тех пор пока не завершится поток, метод которого был вызван

У метода `join()` есть аргумент `timeout` - По умолчанию None, можно передать в него float

In [9]:
def myfunc(a, b):
    time.sleep(2.5)
    print('сумма :', a + b)
    
thr1 = Thread(target = myfunc, args = (1, 2), daemon=True)
thr1.start()
thr1.join(0.125)

if thr1.is_alive():
    print('поток не успел завершиться')
else:
    print('вычисления завершены')

поток не успел завершиться
сумма : 3


### Остановка потока

In [75]:
stop = False 

def myfunc():
    global stop
    while stop == False:
        pass
        
thr1 = Thread(target = myfunc) 
thr1.start() 

stop = True

while thr1.is_alive() == True: 
    pass
print('поток завершился')

поток завершился


### Задача 3 - запись в БД

In [15]:
import sqlite3

db_file = "my_database.db"
table_name = 'my_table'

def recreate_table():
    conn_main = sqlite3.connect(db_file)
    cursor_main = conn_main.cursor()
    cursor_main.execute("DROP TABLE IF EXISTS my_table")
    cursor_main.execute("CREATE TABLE my_table (thread_id INTEGER, value TEXT)")
    conn_main.commit()
    conn_main.close()
    print('table recreated')


def worker_thread(thread_id):
    # Each thread gets its own connection
    conn = sqlite3.connect(db_file, check_same_thread=False)
    cursor = conn.cursor()

    try:
        # Perform database operations
        cursor.execute(f"INSERT INTO my_table (thread_id, value) VALUES (?, ?)", (thread_id, f"data_from_thread_{thread_id}"))
        conn.commit()
        print(f"Thread {thread_id}: Data inserted.")
    except Exception as e:
        print(f"Thread {thread_id}: Error - {e}")
        conn.rollback()
    finally:
        conn.close()


@timer
def threaded3(n):
    threads = []
    for i in range(n):
        thread = Thread(target=worker_thread, args=(i,))
        threads.append(thread)
        thread.start()

    for thread in threads:
        thread.join()

    print("All threads finished.")


recreate_table()
threaded3(555)

table recreated
Thread 0: Data inserted.
Thread 77: Data inserted.
Thread 125: Data inserted.
Thread 177: Data inserted.
Thread 230: Data inserted.
Thread 271: Data inserted.
Thread 288: Data inserted.
Thread 328: Data inserted.
Thread 362: Data inserted.
Thread 407: Data inserted.
Thread 471: Data inserted.
Thread 520: Data inserted.
Thread 13: Data inserted.
Thread 380: Data inserted.
Thread 102: Data inserted.
Thread 62: Data inserted.
Thread 168: Data inserted.
Thread 316: Data inserted.
Thread 448: Data inserted.
Thread 504: Data inserted.
Thread 228: Data inserted.
Thread 70: Data inserted.
Thread 4: Data inserted.
Thread 148: Data inserted.
Thread 120: Data inserted.
Thread 534: Data inserted.
Thread 88: Data inserted.
Thread 313: Data inserted.
Thread 69: Data inserted.
Thread 160: Data inserted.
Thread 232: Data inserted.
Thread 182: Data inserted.
Thread 35: Data inserted.
Thread 14: Data inserted.
Thread 94: Data inserted.
Thread 218: Data inserted.
Thread 522: Data inserted

In [16]:
@timer
def synced3(n):

    for i in range(n):
        conn = sqlite3.connect(db_file)
        cursor = conn.cursor()
    
        try:
            # Perform database operations
            cursor.execute(f"INSERT INTO my_table (thread_id, value) VALUES (?, ?);", (i, f"data_from_thread_{i}"))
            conn.commit()
            print(f"Iter {i}: Data inserted.")
        except Exception as e:
            print(f"Iter {i}: Error - {e}")
            conn.rollback()
        finally:
            conn.close()
    
    print("All finished.")


recreate_table()
synced3(555)

table recreated
Iter 0: Data inserted.
Iter 1: Data inserted.
Iter 2: Data inserted.
Iter 3: Data inserted.
Iter 4: Data inserted.
Iter 5: Data inserted.
Iter 6: Data inserted.
Iter 7: Data inserted.
Iter 8: Data inserted.
Iter 9: Data inserted.
Iter 10: Data inserted.
Iter 11: Data inserted.
Iter 12: Data inserted.
Iter 13: Data inserted.
Iter 14: Data inserted.
Iter 15: Data inserted.
Iter 16: Data inserted.
Iter 17: Data inserted.
Iter 18: Data inserted.
Iter 19: Data inserted.
Iter 20: Data inserted.
Iter 21: Data inserted.
Iter 22: Data inserted.
Iter 23: Data inserted.
Iter 24: Data inserted.
Iter 25: Data inserted.
Iter 26: Data inserted.
Iter 27: Data inserted.
Iter 28: Data inserted.
Iter 29: Data inserted.
Iter 30: Data inserted.
Iter 31: Data inserted.
Iter 32: Data inserted.
Iter 33: Data inserted.
Iter 34: Data inserted.
Iter 35: Data inserted.
Iter 36: Data inserted.
Iter 37: Data inserted.
Iter 38: Data inserted.
Iter 39: Data inserted.
Iter 40: Data inserted.
Iter 41: D

# concurrent.futures

In [2]:
from concurrent.futures import ThreadPoolExecutor

In [3]:
with ThreadPoolExecutor(max_workers=2) as executor:
    future = executor.submit(pow, 5, 2)
    print(future.result())

25


In [23]:
from concurrent.futures import as_completed
import urllib

# Извлечение контента одной страницы
def load_url(url, timeout):
    with urllib.request.urlopen(url, timeout=timeout) as conn:
        return conn.read()

@timer
def threaded3():
    results = []
    # Использование контекстного менеджера
    with ThreadPoolExecutor(max_workers=10) as executor:
        # Start the load operations and mark each future with its URL
        futures = [executor.submit(load_url, url, 60) for url in urls]
        for future in as_completed(futures):
            future_data = future.result()
            results.append(future_data)
    
    return info2(results)   

threaded3()

results: 10, total lenght: 603371
Time: 0.29320 sec (threaded3)


#### Мэппинг вхдящих к выходящим результатам

In [24]:
@timer
def threaded41():
    # Использование контекстного менеджера
    with ThreadPoolExecutor(max_workers=5) as executor:
        # Start the load operations and mark each future with its URL
        future_to_url = {executor.submit(load_url, url, 60): url for url in urls}
        for future in as_completed(future_to_url):
            url = future_to_url[future]
            try:
                data = future.result()
            except Exception as exc:
                print('%r generated an exception: %s' % (url, exc))
            else:
                print('%r page is %d bytes' % (url, len(data)))

threaded41()

'https://docs.python.org/3/tutorial/index.html' page is 35586 bytes
'https://docs.python.org/3/' page is 17833 bytes
'https://docs.python.org/3/library/index.html' page is 75846 bytes
'http://www.python.org' page is 49790 bytes
'https://docs.python.org/3/whatsnew/3.7.html' page is 319290 bytes
'https://docs.python.org/3/reference/index.html' page is 26263 bytes
'https://docs.python.org/3/using/index.html' page is 19342 bytes
'https://docs.python.org/3/howto/index.html' page is 17002 bytes
'https://docs.python.org/3/distributing/index.html' page is 11421 bytes
'https://docs.python.org/3/installing/index.html' page is 30998 bytes
Time: 0.50874 sec (threaded41)


#### Без использования контектснго менеджера


In [25]:
def worker():
    print("Worker thread running")

pool = ThreadPoolExecutor(max_workers=2)

pool.submit(worker)
pool.submit(worker)

pool.shutdown(wait=True)

print("Main thread continuing to run")

Worker thread running
Worker thread running
Main thread continuing to run
